Assamese -custom analyzer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
!pip install emot --upgrade
# to convert emojis to text
!pip install emoji
#to expand a contracted words
!pip install demoji
!pip install contractions

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
import emot
import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
import emoji
import demoji
import contractions
import nltk
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option("max_colwidth" ,220)
wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stopwords = stopwords.words("english")
with open('/content/drive/MyDrive/hasoc/task4/assam/assamese.txt', encoding = 'utf-8') as f:
    assam_stopwords = f.readlines()
    for i in range(len(assam_stopwords)):
        assam_stopwords[i] = re.sub('\n','', assam_stopwords[i])
stopwords = english_stopwords + assam_stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
len(stopwords)

295

In [ ]:
demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

<ipython-input-7-e00aa54160fe>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
demoji.download_codes()
def emo(text):
  try:
   temp=emoji.demojize(str(text),delimiters=(" "," "))
   temp=temp.replace("_","  ")
   return temp
  except IndexError:
        return ""

<ipython-input-8-82a5fb4117d1>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
punct="!#$%&\'()*+,-.’’/:;<=>?@[\\]^_`{|}~’“‘"

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', str(text))

In [ ]:
def clean_text(text):
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))#to remove @ and its following word
    text = contractions.fix(text, slang=True)
    text = "".join([word.lower() for word in text if word not in string.punctuation]) #to remove punctuation
    text="".join([word.lower() for word in text if word not in punct])
    text = "".join([word for word in text if not word.isdigit()])#to remove digit
    text = " ".join(word for word in text.split() if word not in stopwords)
    return text

In [ ]:
asa_train=pd.read_csv('/content/drive/MyDrive/hasoc/task4/assam/train_A_AH_HASOC2023.csv')

In [ ]:
len(asa_train)

4036

In [ ]:
asa_train.shape

(4036, 3)

In [ ]:
asa_train['task_1'].value_counts()

HOF    2347
NOT    1689
Name: task_1, dtype: int64

In [ ]:
asa_train['clean_text'] = asa_train['text'].apply(lambda x:emo(x))
asa_train['clean_text'] = asa_train['clean_text'].apply(lambda x:remove_urls(x))
asa_train['clean_text'] = asa_train['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
asa_train['clean_text']

0                                                                                                                                      লাগিলে ইযাৰ পৰাই কলি লাগাম
1                                                                         আমি গেদা মই গেদা rolling floor laughing rolling floor laughing rolling floor laughing ।
2                                      মানুহে যেতিয়া নিজৰ জীৱন টো নিজৰ কাৰণে সমাজৰ কাৰণে দেশৰ কাৰণে পৃথিৱীৰ কাৰণে কিমান মূল্যবান বুজি নাপায় তেতিয়া এনেকুৱা হয়
3                                                                                                                   কতি খন কোন দিনা দেখা পাও তাকে বাত চায় আছো মই
4       কাপোৰ খন চাইকেল অহা মানুহজনৰ মুখত দলিয়াব লাগেনে বেছেৰাটো গুচি গল সেইকাৰণে বেয়া পায়। rolling floor laughing rolling floor laughing rolling floor laughing
                                                                                  ...                                                                            
4031                        

In [ ]:
#To split train and test
X_train, X_test, y_train, y_test = train_test_split(asa_train.clean_text, asa_train.task_1, test_size=0.25, random_state=42)

In [ ]:
X_train.shape, X_test.shape

((3027,), (1009,))

In [ ]:
import regex
def custom_analyzer(text):
    words = regex.findall(r'\w{2,}', text) #extract words of at least 2 letters
#     for w in words:
    for w in words:
      yield w

In [ ]:
#feature extraction
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer=custom_analyzer,)#range unigram
tfidf_vect.fit(X_train)
xtrain_tfidf =  tfidf_vect.transform(X_train)
xtest_tfidf =  tfidf_vect.transform(X_test)

In [ ]:
type(xtrain_tfidf)

scipy.sparse._csr.csr_matrix

In [ ]:
xtrain_tfidf.shape,xtest_tfidf.shape

((3027, 8498), (1009, 8498))

In [ ]:
type(xtrain_tfidf)

scipy.sparse._csr.csr_matrix

In [ ]:
from sklearn.svm import SVC
svm = SVC()

svm .fit(xtrain_tfidf, y_train)

y_pred = svm.predict(xtest_tfidf)

accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred))


Test Accuracy: 65.5104

               precision    recall  f1-score   support

         HOF       0.64      0.89      0.75       576
         NOT       0.70      0.34      0.46       433

    accuracy                           0.66      1009
   macro avg       0.67      0.62      0.60      1009
weighted avg       0.67      0.66      0.62      1009



In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC

clf1 = LogisticRegression(random_state=1)
clf2 = BernoulliNB()
clf3 = LinearSVC(penalty='l2',C=1.0)

eclf = VotingClassifier(estimators=[('LR', clf1), ('BNB', clf2),
('SVC', clf3)], voting='hard')
eclf = eclf.fit(xtrain_tfidf, y_train)
y_pred = eclf.predict(xtest_tfidf)
accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred))

Test Accuracy: 65.6095

               precision    recall  f1-score   support

         HOF       0.66      0.83      0.73       576
         NOT       0.65      0.42      0.51       433

    accuracy                           0.66      1009
   macro avg       0.66      0.63      0.62      1009
weighted avg       0.66      0.66      0.64      1009

